## GED conflict data clean up

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
##set a working directory
user = os.path.expanduser('~')
display(user)

os.makedirs(f'{user}/Desktop/thesis/', exist_ok=True)
path = os.chdir(f'{user}/Desktop/'+'thesis/')

'/Users/NatStrom'

In [3]:
data = pd.read_csv('GEDEvent_v22_1.csv')

/var/folders/3l/pgxm2f9j3bbgbjcj2pyxl__00000gp/T/ipykernel_18587/2663555733.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('GEDEvent_v22_1.csv')


In [4]:
df = data.copy()

In [5]:
print(df.dtypes)
print(df.columns)
print(df.describe(include='all'))

## sanity check to ensure the spread of the data is correct
print('sanity check 1, ensure appropriate years and countries are represented')
list_country=list(list(df['country_id'].drop_duplicates()))
list_year=list(list(df['year'].drop_duplicates()))
print('country list', list_country)
print('year list', list_year)
#inspect for NAs
print('following checks for completeness of all columns')
print(df['country_id'].isnull().values.any())

id                     int64
relid                 object
year                   int64
active_year            int64
code_status           object
type_of_violence       int64
conflict_dset_id       int64
conflict_new_id        int64
conflict_name         object
dyad_dset_id           int64
dyad_new_id            int64
dyad_name             object
side_a_dset_id         int64
side_a_new_id          int64
side_a                object
side_b_dset_id         int64
side_b_new_id          int64
side_b                object
number_of_sources      int64
source_article        object
source_office         object
source_date           object
source_headline       object
source_original       object
where_prec             int64
where_coordinates     object
where_description     object
adm_1                 object
adm_2                 object
latitude             float64
longitude            float64
geom_wkt              object
priogrid_gid           int64
country               object
country_id    

In [6]:
#filter only the wanted columns
df_filtered = df.filter(['id', 'relid', 'year', 'type_of_violence','conflict_new_id','country_id','best', 'high', 'low'])

#df_filtered = 
#add the iso codes


In [7]:
display(df_filtered)

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low
0,244657,IRQ-2017-1-524-322,2017,1,259,700,6,6,6
1,412700,IRQ-2021-1-524-145,2021,1,259,700,183,184,171
2,413023,IRQ-2021-1-524-143,2021,1,259,700,2,3,0
3,412909,IRQ-2021-1-524-144,2021,1,259,700,10,10,9
4,132140,AFG-1989-1-411-2,1989,1,333,700,6,6,6
...,...,...,...,...,...,...,...,...,...
293629,15525,MZM-1989-3-1347-4,1989,3,562,552,9,9,9
293630,15524,MZM-1989-3-1347-16,1989,3,562,552,9,9,9
293631,15245,MZM-1990-3-1347-18,1990,3,562,552,7,7,7
293632,15516,MZM-1990-3-1347-28,1990,3,562,552,1,1,1


In [8]:
from ingester3.scratch import cache_manager

cache_manager()

# This will import the Pandas extensions as well as bootstrap the object layer
from ingester3.extensions import *

In [9]:
#create the c_id from the gw codes
df_filtered = pd.DataFrame.c.from_gwcode(df_filtered, gw_col='country_id', month_col=None)

In [24]:
df_conflict = df_filtered.copy()

In [25]:
df_conflict = df_conflict.filter(['c_id','year', 'type_of_violence', 'best', 'high', 'low'])
df_conflict

,c_id,year,type_of_violence,best,high,low
0,133,2017,1,6,6,6
1,133,2021,1,183,184,171
2,133,2021,1,2,3,0
3,133,2021,1,10,10,9
4,133,1989,1,6,6,6
...,...,...,...,...,...,...
293629,158,1989,3,9,9,9
293630,158,1989,3,9,9,9
293631,158,1990,3,7,7,7
293632,158,1990,3,1,1,1


In [26]:
#replace row values
df_conflict['type_of_violence'] = df_conflict['type_of_violence'].map({1:'sb',
                                            2:'ns',
                                            3:'os'})

In [27]:
#create dummies
def categorize(row):  
    if row['type_of_violence'] == 'sb':
        return 1
    return 0

df_conflict['dummy_sb'] = df_conflict.apply(lambda row: categorize(row), axis=1)

# repeat for non-state violence
def categorize(row):  
    if row['type_of_violence'] == 'ns':
        return 1
    return 0

df_conflict['dummy_ns'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for one-sided violence
def categorize(row):  
    if row['type_of_violence'] == 'os':
        return 1
    return 0

df_conflict['dummy_os'] = df_conflict.apply(lambda row: categorize(row), axis=1)



In [28]:
#add dummies for the type of violence
df_conflict

,c_id,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os
0,133,2017,sb,6,6,6,1,0,0
1,133,2021,sb,183,184,171,1,0,0
2,133,2021,sb,2,3,0,1,0,0
3,133,2021,sb,10,10,9,1,0,0
4,133,1989,sb,6,6,6,1,0,0
...,...,...,...,...,...,...,...,...,...
293629,158,1989,os,9,9,9,0,0,1
293630,158,1989,os,9,9,9,0,0,1
293631,158,1990,os,7,7,7,0,0,1
293632,158,1990,os,1,1,1,0,0,1


In [29]:
#only filter relevant c_ids
df_conflict.query('c_id == 40 or c_id == 41 or c_id == 42 or c_id == 43 or c_id ==  47 or c_id ==  48 or c_id ==  49 or c_id ==  50 or c_id ==  52 or c_id ==  53 or c_id ==  54 or c_id ==  55 or c_id ==   56 or c_id ==   57 or c_id ==   69 or c_id ==   70 or c_id ==   74 or c_id ==   76 or c_id ==   78 or c_id ==   79 or c_id ==   80 or c_id ==   81 or c_id ==  120 or c_id ==  154 or c_id ==  155 or c_id ==  156 or c_id ==  157 or c_id ==  158 or c_id ==  159 or c_id ==  160 or c_id ==  161 or c_id ==  162 or c_id ==  163  or c_id ==  164or c_id ==   165 or c_id ==  166 or c_id ==  169 or c_id ==  170 or c_id ==  172 or c_id ==   191 or c_id ==  214 or c_id ==  235 or c_id ==  237 or c_id ==   244 or c_id ==  246')

,c_id,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os
45621,165,1989,sb,1,1,1,1,0,0
45622,165,1989,sb,0,16,0,1,0,0
45623,165,1989,sb,0,6,0,1,0,0
45624,165,1989,sb,0,13,0,1,0,0
45625,165,1989,sb,0,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...
293629,158,1989,os,9,9,9,0,0,1
293630,158,1989,os,9,9,9,0,0,1
293631,158,1990,os,7,7,7,0,0,1
293632,158,1990,os,1,1,1,0,0,1


In [30]:
#create dummies for intensity
def categorize(row):  
    if row['best'] <= 100:
        return 'low'
    if row['best'] > 100 or row['best'] < 1000:
        return 'medium'
    return 'high'

df_conflict['intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)
df_conflict

,c_id,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os,intensity
0,133,2017,sb,6,6,6,1,0,0,low
1,133,2021,sb,183,184,171,1,0,0,medium
2,133,2021,sb,2,3,0,1,0,0,low
3,133,2021,sb,10,10,9,1,0,0,low
4,133,1989,sb,6,6,6,1,0,0,low
...,...,...,...,...,...,...,...,...,...,...
293629,158,1989,os,9,9,9,0,0,1,low
293630,158,1989,os,9,9,9,0,0,1,low
293631,158,1990,os,7,7,7,0,0,1,low
293632,158,1990,os,1,1,1,0,0,1,low


In [31]:
# create dummy for low intensity conflict
def categorize(row):  
    if row['intensity'] == 'low':
        return 1
    return 0

df_conflict['dummy_low_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for medium intensity
def categorize(row):  
    if row['intensity'] == 'middle':
        return 1
    return 0

df_conflict['dummy_medium_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

def categorize(row):  
    if row['intensity'] == 'high':
        return 1
    return 0

df_conflict['dummy_high_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

In [32]:
df_conflict = df_conflict.loc[df_conflict['year'] <= 2019]

In [34]:
df_conflict = df_conflict.loc[df_conflict['year'] >= 1990]

In [35]:
df_conflict

,c_id,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os,intensity,dummy_low_intensity,dummy_medium_intensity,dummy_high_intensity
0,133,2017,sb,6,6,6,1,0,0,low,1,0,0
55,133,1990,sb,450,1100,450,1,0,0,medium,0,0,0
56,133,1990,sb,20,20,20,1,0,0,low,1,0,0
57,133,1990,sb,8,8,0,1,0,0,low,1,0,0
58,133,1990,sb,15,15,0,1,0,0,low,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293626,158,2019,os,1,1,1,0,0,1,low,1,0,0
293627,158,2019,os,2,2,2,0,0,1,low,1,0,0
293631,158,1990,os,7,7,7,0,0,1,low,1,0,0
293632,158,1990,os,1,1,1,0,0,1,low,1,0,0


In [36]:
df_conflict = df_conflict.groupby(['c_id','year']).reset_index()
df_conflict

AttributeError: 'DataFrameGroupBy' object has no attribute 'reset_index'

In [22]:
df_conflict.reset_index()

AttributeError: 'DataFrameGroupBy' object has no attribute 'reset_index'

In [37]:
df_conflict.to_csv('df_conflict.csv')